# Parse LaTeX file and convert into DeepL-friendly format

## Input LaTeX source from file and parse

First replace newly defined commands by original commands to help pylatexenc parsing well

In [1]:
import re
def replaceNewCommand(orig_command, match_command):
    return orig_command + match_command.group(1)

def preprocessing(orig):
    mod = orig
    while True:
        match = re.search(r'\\newcommand\{(.*?)\}\{(.*)\}', mod)
        if match is None:
            break
        mod = mod.replace(match.group(), '')
        mod = re.sub(match.group(1).replace('\\','\\\\')+'([\\\\ \n\{\}\(\)\^\$_\+\-\*\/=,.])', lambda wrapper: replaceNewCommand(match.group(2), wrapper), mod)
    while True:
        match = re.search(r'\\def([^\#\n]*?)\{(.*)\}', mod)
        if match is None:
            break
        mod = mod.replace(match.group(), '')
        mod = re.sub(match.group(1).replace('\\','\\\\')+'([\\\\ \n\{\}\(\)\^\$_\+\-\*\/=,.])', lambda wrapper: replaceNewCommand(match.group(2), wrapper), mod)
    return mod

In [2]:
from pylatexenc.latexwalker import LatexWalker, LatexCharsNode, LatexCommentNode, LatexEnvironmentNode, LatexGroupNode, LatexMacroNode, LatexMathNode, LatexSpecialsNode

input_file = input()
with open(input_file) as f:
    latex_orig = f.read()
#############################################
# latex_mod = preprocessing(latex_orig)
latex_mod = latex_orig
#############################################
w = LatexWalker(latex_mod)
(nodelist, pos, len_) = w.get_latex_nodes(pos=0)

 v0.tex


## Look for \begin{document} ... \end{document} environment

Tempolary: search for the document environment step by step

In [3]:
def searchDocument(nl):
    for i in range(len(nl)):
        if nl[i] is None:
            continue
        if nl[i].isNodeType(LatexEnvironmentNode) and nl[i].environmentname == 'document':
            return nl[i]
            # return [i]
        else:
            if nl[i].isNodeType(LatexCharsNode) or nl[i].isNodeType(LatexCommentNode) or nl[i].isNodeType(LatexSpecialsNode):
                continue
            elif nl[i].isNodeType(LatexMacroNode):
                if nl[i].nodeargd is None:
                    continue
                else:
                    next = nl[i].nodeargd.argnlist
            else:
                next = nl[i].nodelist
            res = searchDocument(next)
            if res != []:
                return res
                # return [i,res]
    return []

In [4]:
doc = searchDocument(nodelist)
# env = [n for n in nodelist if n.isNodeType(LatexEnvironmentNode)]
# doc = [e for e in env if e.environmentname=='document']
# if len(doc)==1:
#    doc = doc[0]
#else:
#    print('Unexpected format with more than one document environment')

## Look for successive LaTeX special expressions and replace them by the format N(from)T(to)

In [5]:
def replaceSpecial(i, node):
    if node.isNodeType(LatexCharsNode):
        s = node.chars
#    elif node.isNodeType(LatexCommentNode):
#        s = ''
    elif node.isNodeType(LatexSpecialsNode) and node.specials_chars == '~':
        s = ' '
    else:
        s = ' N'+str(i)+'T'+str(i)+' '
    return s

In [6]:
def replaceSuccessiveTags(match):
    return 'N'+match.group(1)+'T'+match.group(2)

In [7]:
dnl = doc.nodelist
str_list = [replaceSpecial(i, n) for i, n in enumerate(dnl)]
latex_rep = ''.join(str_list)
while re.search('N\d+T\d+ [ \n]* N\d+T\d+', latex_rep) is not None:
    latex_rep = re.sub('N(\d+)T\d+ [ \n]* N\d+T(\d+)', replaceSuccessiveTags, latex_rep)

## Preprocess the string and translate using DeepL

In [8]:
paras = re.split('\n\n', latex_rep)
paras = [p.replace('\n', ' ') for p in paras]

In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import time
import pyperclip as ppc

options = Options()
options.add_argument('--disable-gpu')
options.add_argument('--disable-extensions')
options.add_argument('--proxy-server="direct://"')
options.add_argument('--proxy-bypass-list=*')
options.add_argument('--start-maximized')

DRIVER_PATH = '/usr/local/bin/chromedriver'
driver = webdriver.Chrome(executable_path=DRIVER_PATH, options=options)

load_url = 'https://www.deepl.com/ja/translator#en/ja'
driver.get(load_url)

clipboard = ppc.paste()
stextarea = driver.find_element_by_css_selector(
    '.lmt__textarea.lmt__source_textarea.lmt__textarea_base_style')
ttextarea = driver.find_element_by_css_selector(
    '.lmt__textarea.lmt__target_textarea.lmt__textarea_base_style')

In [10]:
def translateParagraph(par):
    if par == '' or re.fullmatch('[ \n]+', par) is not None:
        return par
    ppc.copy(par)
    stextarea.send_keys(Keys.COMMAND, 'v')
    translated_text = ''
    while not translated_text:
        time.sleep(1)
        translated_text = ttextarea.get_property('value')
    stextarea.send_keys(Keys.COMMAND, 'a')
    stextarea.send_keys(Keys.BACKSPACE)
    return translated_text

In [11]:
paras_ja = [translateParagraph(par) for par in paras]
latex_ja = '\n\n'.join(paras_ja)

In [12]:
ppc.copy(clipboard)
driver.quit()

## Debug: comparing original tags and "translated" ones

In [13]:
def node2tag(node):
    return 'N'+node[0]+'T'+node[1]

In [14]:
pattern = re.compile('N(\d+)T(\d+)')
tag0 = pattern.findall(latex_rep)
tag1 = pattern.findall(latex_ja) + re.compile('N\d+T\d+').findall(latex_ja)
tag_miss = list((set(tag0) ^ set(tag1)) & set(tag0))
tag_miss = [node2tag(t) for t in tag_miss]
(tag_miss)

['N914T918',
 'N1040T1040',
 'N942T942',
 'N806T806',
 'N944T947',
 'N664T664',
 'N298T301',
 'N912T912',
 'N122T122',
 'N1011T1011',
 'N510T510',
 'N808T811',
 'N910T910',
 'N296T296',
 'N120T120']

In [25]:
target = [p for p in paras if tag_miss[8] in p]
paras.index(target[0])

3

In [26]:
paras[3]

'In Ref.  N108T108 , we revisited the  N110T110  regime, this time lifting the assumption of efficient thermal contact between the dark sector and the SM. We showed that dark sector processes like  N112T112  (hereafter denoted  N114T114 ) and  N116T116  (hereafter denoted  N118T118 ) can generate or remove a significant amount of heat during freezeout; if the energy transfer rate between the SM and the dark sector is not large enough, this can lead to the dark sector kinetically decoupling from the SM before freezeout is complete, leading to a dark sector temperature  N120T120  that is different from the SM temperature  N122T122 . Kinetic decoupling can occur before freezeout is complete for  N124T124  as large as  N126T126  for  N128T128 , dramatically altering how the correct relic abundance is achieved. While some parameter space in the range  N130T130  remains open, especially for  N132T132 , this model still faces strong constraints from limits on the self-interaction cross sectio

In [27]:
paras_ja[3]

'Ref.  N108T108では、N110T110領域を再検討し、今回は暗黒セクターとSMの間の効率的な熱的接触の仮定を解除しました。N112T112（以下、N114T114と表記）やN116T116（以下、N118T118と表記）のような暗黒セクターのプロセスは、フリーズアウト中にかなりの量の熱を生成または除去することができることを示した。N124T124では凍結が完了する前に運動論的デカップリングが起こり、N128T128ではN126T126となり、正しいレリックアバンダンスを得る方法が劇的に変化する。N130T130の範囲では、特にN132T132のパラメータスペースが残っていますが、このモデルはDMの自己相互作用断面積の制限や前述のCMBパワースペクトルによる強い制約を受けています。\nLatex2DeepL missing expresion warning: "N122T122", "N120T120", '

In [18]:
warning1 = '\nLatex2DeepL missing expresion warning: '
warning2 = ', '
def addMissingExpressionWarning(tag):
    target = [p for p in paras if tag in p]
    index = paras.index(target[0])
    if re.search(warning1, paras_ja[index]) is None:
        paras_ja[index] += warning1+'\"'+tag+'\", '
    else:
        paras_ja[index] += '\"'+tag+'\", '
for t in tag_miss:
    addMissingExpressionWarning(t)

In [19]:
latex_ja = '\n\n'.join(paras_ja)

## Replace Node#(from)to(to) to original LaTeX formula

In [20]:
def outputLatex(nl):
    latex = ''
    if type(nl) is not list:
        nl = [nl]
    for i in range(len(nl)):
        if nl[i] is None:
            continue
        if nl[i].isNodeType(LatexCharsNode):
            latex += nl[i].chars
        #elif nl[i].isNodeType(LatexCommentNode):
        #    print(nl[i])
        elif nl[i].isNodeType(LatexEnvironmentNode):
            evn = nl[i].environmentname
            latex += '\\begin{'+evn+'}'
            if nl[i].nodeargd.argnlist != [] and nl[i].nodeargd.argnlist != [None]:  # e.g., \begin{tabular}{ccc}
                latex += outputLatex(nl[i].nodeargd.argnlist)
            latex += outputLatex(nl[i].nodelist)
            latex += '\\end{'+evn+'}'
        elif nl[i].isNodeType(LatexGroupNode):
            latex += nl[i].delimiters[0]
            latex += outputLatex(nl[i].nodelist)
            latex += nl[i].delimiters[1]
        elif nl[i].isNodeType(LatexMacroNode):
            latex += '\\'+nl[i].macroname
            latex += outputLatex(nl[i].nodeargd.argnlist)
            latex += nl[i].macro_post_space
        elif nl[i].isNodeType(LatexMathNode):
            latex += nl[i].delimiters[0]
            latex += outputLatex(nl[i].nodelist)
            latex += nl[i].delimiters[1]
        elif nl[i].isNodeType(LatexSpecialsNode):
            latex += nl[i].specials_chars
    return latex

In [21]:
def replaceRule(match):
    (fr, to) = (int(match.group(1)), int(match.group(2)))
    return outputLatex(dnl[fr:to+1])

In [22]:
latex_fin = re.sub('N(\d+)T(\d+)', replaceRule, latex_ja)
latex_fin = latex_mod[:doc.pos]+'\\begin{document}\n'+latex_fin+'\n\\end{document}'

In [23]:
(head, ext) = re.split('\.', input_file)
output_file = head+'_ja.'+ext
with open(output_file, mode='w') as f:
    f.write(latex_fin)